# GEOS 505 Project

## 20181129 Assignment Update



The code posted below is a work in progress. I've had to do some scrambling because I wasn't able to download the data I needed due to server problems.

I'll update the project update later with more details.

In order to get this far I had to do several steps:

I downloaded large DEMs from https://earthdata.nasa.gov/ then I made a mosaic of them in arcGIS because I did find an easy way to do it in python. 

Using the DEMs I delineated several watersheds in the Main Salmon river Drainage.

I attempted to download NDSI data for the (large) area the NASA processing servers failed. At this point I decided to use data I previously downloaded for a different project. Because that data covered a different area, I delineated the Payette River Drainage at Horseshoe Bend Id.

The next step was to resample the DEM data to 500x500 meter resolution in order to match the NDSI_Snow_Cover data. I also did this in ArcGIS although if I had more time, I would have preferred to do it in Python.

Next, using python I clipped the NDSI data and the resampled DEM to the watershed drainage and created numpy arrays from the data.

The next steps are to break the elevations in to groups.
Determine % snow cover in each elevation band (group) for each day of the year.
Create hydrograph(s).
Plot time vs elevation band snow coverage and a hydrograph for a given year.




In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 28 13:15:24 2018

@author: steve
"""

import numpy as np
import matplotlib.pyplot as plt
import glob
import gdal
import datetime
import fiona
import rasterio
import rasterio.mask
#from PIL import Image

#import pandas as pd

# This line can be used to convert a date to a day number.
# Enter the analysis date below:
daynumber = datetime.datetime(2001, 1, 1).timetuple().tm_yday


# Path to directory containing hdf files
HDF_file_path = '/Users/steve/Documents/classes/Geos_505/project_Payette/working/data_files/MODIS_NSDI/'

# Path to write tif images
file_out_path = '/Users/steve/Documents/classes/Geos_505/project_Payette/working/5_images/'

# Path to clipped NDSI_Snow_Cover tifs
clippedDir = '/Users/steve/Documents/classes/Geos_505/project_Payette/working/6_clipped_SC/'

# Use the variable below later to parse out the file date.
file_path_len = len(HDF_file_path)


# Some variables to extract the datasubset we need from the HDF
datasub1 = 'HDF4_EOS:EOS_GRID:"'
datasub2 = '":MOD_Grid_Snow_500m_16:NDSI_Snow_Cover'
        


# Get a list of all the .hdf files in the directory
# then find the number of images
print('\n\nScanning directory...\n',HDF_file_path)
MODIS_files = glob.glob(HDF_file_path + '*.hdf')



x = 0   
print ('Files will be written to: ', clippedDir)
# Answer yes to create Snow Proability images
write_images = input('Do you want to create '+ str(np.size(MODIS_files)) + 'clipped NDSI_Snow_Cover images? [y/n]  ')
if write_images.lower() =='yes'or write_images.lower() =='y':

    for file_name in MODIS_files:
        date = file_name[file_path_len+ 9:file_path_len+ 16]
            
        rasterio.open(datasub1+str(file_name)+datasub2)
        
        
        with fiona.open('/Users/steve/Documents/classes/Geos_505/project_Payette/working/data_files/Payette_drain_shapeFile/Payette_ply.shp', 'r') as shapefile:
            features = [feature['geometry'] for feature in shapefile]
            
           
        with rasterio.open(datasub1+str(file_name)+datasub2) as src:
            out_image, out_transform = rasterio.mask.mask(src, features,
                                                                crop=True)
            out_meta = src.meta.copy()
            
        
        out_meta.update({'driver': 'GTiff',
                         'height': out_image.shape[1],
                         'width': out_image.shape[2],
                         'transform': out_transform})
       
        print('writing clipped data as: ', date +'.tif')
    
        with rasterio.open(clippedDir + date + '.tif', 'w', **out_meta) as dest:
            dest.write(out_image)
     
        x=x++1
        
    print ('Files written to: ', clippedDir)
    

else:
  print ('Ok, images not created at your request.')

# =============================================================================
# Load the data which has been clipped to a shapefile into an a numpy array
#  
# 
# =============================================================================
ClippedTif_files = glob.glob(clippedDir + '*.tif')

# One sample per year from 18 years of record.
number_of_years = 17

# Get the height and width of the data files from the first image.
# These will e used to initialize the array
dataset = gdal.Open(ClippedTif_files[0], gdal.GA_ReadOnly)
temp_array = dataset.ReadAsArray()
array_size = temp_array.shape
height = array_size[0]
width = array_size[1]


# Check to see if there is an existing numpy array. If file not
# found read through the HDF files in the directory.
database_file_path = '/Users/steve/Documents/classes/Geos_505/project_Payette/working/data_files/'
database_file = database_file_path + 'database_365.npy'
path_len = len(clippedDir)

# =============================================================================
# Troubleshooting...
# #Logoutput to file
# import sys
# sys.stdout = open('/Users/steve/Documents/classes/Geos_505/project_Payette/working/7_plots/log.txt', 'w')
# 
# =============================================================================


i = 0
try:
    with open(database_file) as file:
        pass
except IOError as e:
    ##############################################################
    # The section below loads yearly data by 
    # year then day number. Day number i.e.; day number
    # 73 = March 14.
    # indexes are:
    # NDSI_SC_array [year, day_number, pixel row, pixel column]
    
    # Initialize the main array. Fill with the value -300. This value can
    # be filter out later if needed.
    NDSI_SC_array = np.full((number_of_years,366,height,width), -300, dtype=np.float64)
   

    for file_name in ClippedTif_files:
        file = gdal.Open(file_name, gdal.GA_ReadOnly)
        #image = Image.open(file_name)
        date = file_name[path_len+ 0:path_len+ 7]
        year = int(date[0:4])
        dayoftheyear = int(date[4:7])
        print('\nReading : ',file_name[77:],'\nYear: ',  year, ',  Day of the year : ',dayoftheyear)
        temp = None
        band = file.GetRasterBand(1)
        temp = band.ReadAsArray()
        #temp = numpy.array(image)
        #print(temp)
        NDSI_SC_array[year-2001, dayoftheyear-1] = temp
        i=i++1
                
else:
    print('\n\n###########################')
    print('Loading data from file:\n',database_file)
    print('###########################')
    NDSI_SC_array = np.load(database_file)



if (i==0):
    print('\n\n###########################')
    print('Data loaded from numpy data file.')
    print('###########################')
else :
    print('\n\n###########################')
    print(i,'  HDF files were processed')
    print('###########################')
          
# test
# print(NDSI_SC_array[16,5].mean())

###############################################################################
# The line below can be uncommented to write the data array to a file.
# This will be useful if you don't wish to read the data directly
# from the HDF files later.
# np.save(database_file_path + 'database_365', NDSI_SC_array)
#
# The line below can be uncommented to write the calculated snow percentage array to a file.
# np.save(database_file_path + 'Snow_Percent', Snow_Percent)


######################################################
# Remove values less than 0 and greater than 100
temp_arr = (NDSI_SC_array < 0) | (NDSI_SC_array > 100)
NDSI_SC_array_nan = np.ma.array(NDSI_SC_array, mask=temp_arr, fill_value=np.nan)
Snow_Percent = NDSI_SC_array_nan.filled()

   
# =============================================================================
# Test pot of a day... 
# plt.imshow(Snow_Percent[16,4], aspect='auto', origin='lower')
# plt.savefig('py.png')
# 
# 
# =============================================================================


# =============================================================================
# Clip the DEM to watershed. Note the DEM was resampled to a 500x500m resolution
# to match the MODIS grid size.
# =============================================================================



#########
##### Fix the line below, the DEM must be clipped to a shapefile with the same 
##### dimensions as the Snow Array (make a new shape file in arcGIS that cuts
##### off the top of the drainage grrrr... because my old data doesn't cover
##### the whole area).
#########
with fiona.open('/Users/steve/Documents/classes/Geos_505/project_Payette/working/data_files/Payette_drain_shapeFile/Payette_ply.shp', 'r') as shapefile:
            features = [feature['geometry'] for feature in shapefile]
            
           
with rasterio.open('/Users/steve/Documents/classes/Geos_505/project_Payette/working/data_files/DEM_Mosaic_Resample1.tif') as src:
    out_image, out_transform = rasterio.mask.mask(src, features,
                                                       crop=True)
    out_meta = src.meta.copy()
            
        
out_meta.update({'driver': 'GTiff',
                 'height': out_image.shape[1],
                 'width': out_image.shape[2],
                 'transform': out_transform})
       
with rasterio.open('/Users/steve/Documents/classes/Geos_505/project_Payette/working/data_files/clipped_DEM_Mosaic_Resample1.tif', 'w', **out_meta) as dest:
            dest.write(out_image)
            
# =============================================================================
# Load the elevation data from the clipped DEM
# =============================================================================

file = gdal.Open('/Users/steve/Documents/classes/Geos_505/project_Payette/working/data_files/clipped_DEM_Mosaic_Resample1.tif', gdal.GA_ReadOnly)
elevation = file.GetRasterBand(1)
elevation_arr = elevation.ReadAsArray()


#######################################
#######################################
# =============================================================================
# Next steps break the elevations in to groups.
# Determine % snow cover in each elevation band for each day of the year.
# Create hydrograph
# plot time vs e;evation band snow coverage and hydrograph for given year.
# 
# =============================================================================
























